In [23]:

import os
from eval_loc import API_Evaluator



2024-11-20 16:10:22.420 | INFO     | modelzipper.tutils:auto_read_dir:325 - number of files with prefix '' and suffix 'api': 3
2024-11-20 16:10:22.421 | INFO     | modelzipper.tutils:auto_read_dir:325 - number of files with prefix '' and suffix 'location': 1
2024-11-20 16:10:22.422 | INFO     | modelzipper.tutils:auto_read_dir:325 - number of files with prefix '' and suffix 'location': 1
2024-11-20 16:10:22.423 | INFO     | modelzipper.tutils:auto_read_dir:325 - number of files with prefix '' and suffix 'location': 1
2024-11-20 16:10:22.423 | INFO     | modelzipper.tutils:auto_read_dir:325 - number of files with prefix '' and suffix 'jsonl': 1
2024-11-20 16:10:22.424 | INFO     | modelzipper.tutils:auto_read_dir:325 - number of files with prefix '' and suffix 'jsonl': 1
2024-11-20 16:10:22.425 | INFO     | modelzipper.tutils:auto_read_dir:325 - number of files with prefix '' and suffix 'jsonl': 1
2024-11-20 16:10:22.425 | INFO     | __main__:<module>:10 - ['/mnt/petrelfs/tangzecheng/lo

2024-11-20 16:10:27.885 | INFO     | eval_loc:eval_api_res:19 - load evaluators, rouge and bleu scores


KeyboardInterrupt: 

In [21]:
print(all_content.keys())
print(all_content['preds_rapid_single_api'][0].keys())

dict_keys(['preds_rapid_single_api', 'preds_rapid_parallel_api', 'preds_rapid_multiple_api'])
dict_keys(['bucket_id', 'query', 'message', 'testing_setting', 'dataset_name', 'call_parameters', 'source_docs', 'pred'])


dict_keys(['bucket_id', 'query', 'message', 'testing_setting', 'dataset_name', 'call_parameters', 'source_docs', 'pred'])